In [ ]:
# Steps to install
# 1. pip install sqlalchemy-bigquery google-cloud-bigquery-storage pyarrow
# 2. Copy the credentials file to wherever you set BIGQUERY_CREDENTIALS_PATH to

In [11]:
import json
import os
import pandas as pd
import pyarrow

import sys
from os import path
import numpy

from dotenv import load_dotenv
from sqlalchemy import create_engine


load_dotenv(verbose=True)
BIGQUERY_CREDENTIALS_PATH = os.environ.get('BIGQUERY_CREDENTIALS_PATH')

In [12]:
sys.path.append(path.realpath(path.join(os.getcwd(), "../core")))
import create_queries
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
import prepare_data

In [14]:
COMPANY_NAME = 'DP'
COMPANY_IDENTIFIER = 'DP'
TRANSFER_PACKAGES_START_DATE = '2020-01-01'
SALES_TRANSACTIONS_START_DATE = '2020-01-01'

INVENTORY_DATE = '10/28/2021'

In [15]:
company_incoming_transfer_packages_query = create_queries.create_company_incoming_transfer_packages_query(COMPANY_IDENTIFIER, TRANSFER_PACKAGES_START_DATE)
company_outgoing_transfer_packages_query = create_queries.create_company_outgoing_transfer_packages_query(COMPANY_IDENTIFIER, TRANSFER_PACKAGES_START_DATE)
company_sales_transactions_query = create_queries.create_company_sales_transactions_query(COMPANY_IDENTIFIER, SALES_TRANSACTIONS_START_DATE)
company_inventory_packages_query = create_queries.create_company_inventory_packages_query(COMPANY_IDENTIFIER)

engine = create_engine('bigquery://bespoke-financial/ProdMetrcData', credentials_path=os.path.expanduser(BIGQUERY_CREDENTIALS_PATH))
company_incoming_transfer_packages_dataframe = pd.read_sql_query(company_incoming_transfer_packages_query, engine)
company_outgoing_transfer_packages_dataframe = pd.read_sql_query(company_outgoing_transfer_packages_query, engine)
company_sales_transactions_dataframe = pd.read_sql_query(company_sales_transactions_query, engine)
company_inventory_packages_dataframe = pd.read_sql_query(company_inventory_packages_query, engine)

In [6]:
deduped_sales_receipts_dataframe = prepare_data.dedupe_sales_transactions(company_sales_transactions_dataframe)

In [16]:
df_in = company_incoming_transfer_packages_dataframe
df_out = company_outgoing_transfer_packages_dataframe
df_inventory = company_inventory_packages_dataframe

In [7]:

df_sales = deduped_sales_receipts_dataframe


In [17]:
pd.options.display.float_format = '{:,.2f}'.format

## Sales

In [9]:
df_sales.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 17 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   id                        0 non-null      object
 1   license_number            0 non-null      object
 2   receipt_number            0 non-null      object
 3   rt_type                   0 non-null      object
 4   sales_customer_type       0 non-null      object
 5   sales_datetime            0 non-null      object
 6   total_packages            0 non-null      object
 7   rt_total_price            0 non-null      object
 8   tx_type                   0 non-null      object
 9   tx_package_id             0 non-null      object
 10  tx_package_label          0 non-null      object
 11  tx_product_name           0 non-null      object
 12  tx_product_category_name  0 non-null      object
 13  tx_unit_of_measure        0 non-null      object
 14  tx_quantity_sold          0 non-null      o

In [143]:
# df_sales.to_csv('./MW_sales.csv')

In [10]:
df_sales['per_unit'] = df_sales['tx_total_price'] / df_sales['tx_quantity_sold']
df_sales['year_month'] = df_sales['sales_datetime'].dt.strftime("%Y-%m")

In [11]:
df_sales.groupby('year_month').sum()

,total_packages,rt_total_price,tx_quantity_sold,tx_total_price,per_unit
year_month,,,,,
2020-01,226125,"5,514,642.60","176,257.82","2,244,143.75","1,476,546.14"
2020-02,257748,"5,765,387.15","173,109.34","2,299,761.12","1,538,756.27"
2020-03,272156,"6,831,152.88","184,994.58","2,589,911.90","1,786,150.29"
2020-04,255702,"6,865,193.66","151,115.06","2,432,639.72","1,871,813.53"
2020-05,274049,"7,848,426.37","179,104.35","2,874,144.58","2,252,517.84"
2020-06,285761,"7,889,127.54","157,793.25","2,865,584.60","2,297,075.57"
2020-07,348676,"8,846,268.60","160,140.00","3,108,843.55","2,612,570.00"
2020-08,389810,"8,765,135.40","161,643.75","3,018,253.05","2,553,867.57"
2020-09,376265,"8,535,706.31","156,742.00","2,960,355.28","2,521,820.38"


In [12]:
s_total_count = df_sales.groupby('year_month')['tx_total_price'].count()
df_total_count = pd.Series(s_total_count).to_frame()
df_total_count = df_total_count.reset_index()
df_total_count.rename(columns={'tx_total_price':'total_count'}, inplace=True)

In [13]:
s_revenue = df_sales.groupby('year_month')['tx_total_price'].sum()
df_revenue = pd.Series(s_revenue).to_frame()
df_revenue = df_revenue.reset_index()
df_revenue.rename(columns={'tx_total_price': 'revenue'}, inplace=True)

In [14]:
df_revenue

,year_month,revenue
0,2020-01,"2,244,143.75"
1,2020-02,"2,299,761.12"
2,2020-03,"2,589,911.90"
3,2020-04,"2,432,639.72"
4,2020-05,"2,874,144.58"
5,2020-06,"2,865,584.60"
6,2020-07,"3,108,843.55"
7,2020-08,"3,018,253.05"
8,2020-09,"2,960,355.28"
9,2020-10,"2,929,401.65"


### Outgoing

In [18]:
df_out.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66022 entries, 0 to 66021
Data columns (total 28 columns):
 #   Column                                 Non-Null Count  Dtype              
---  ------                                 --------------  -----              
 0   package_row_id                         66022 non-null  object             
 1   delivery_type                          66022 non-null  object             
 2   license_number                         66022 non-null  object             
 3   manifest_number                        66022 non-null  object             
 4   created_date                           66022 non-null  object             
 5   received_datetime                      65104 non-null  datetime64[ns, UTC]
 6   shipper_facility_license_number        66022 non-null  object             
 7   shipper_facility_name                  66022 non-null  object             
 8   recipient_facility_license_number      66022 non-null  object             
 9   recipi

In [19]:
df_out['per_unit'] = df_out['shipper_wholesale_price'] / df_out['shipped_quantity']
df_out['year_month'] = df_out['received_datetime'].dt.strftime("%Y-%m")

In [20]:
df_out.groupby('year_month').sum()

,is_testing_sample,is_trade_sample,shipper_wholesale_price,shipped_quantity,received_quantity,item_unit_weight,per_unit
year_month,,,,,,,
2020-11,0,299,"337,650.75","101,585.00","100,895.00","48,505.60","10,677.27"
2020-12,0,553,"435,871.85","145,320.00","144,678.00","54,761.50","11,087.69"
2021-01,0,199,"736,300.95","161,942.00","159,389.00","52,942.30","17,314.72"
2021-02,0,218,"457,395.03","145,588.00","143,323.00","47,970.70","11,404.14"
2021-03,0,432,"505,702.80","174,411.00","172,061.00","55,205.20","12,054.81"
2021-04,0,595,"778,365.67","245,143.00","241,271.00","77,041.70","17,982.47"
2021-05,3,931,"635,557.14","210,302.00","208,007.00","70,137.60","15,680.33"
2021-06,26,964,"711,308.80","234,790.25","233,370.25","76,048.70","17,228.76"
2021-07,38,1608,"952,267.20","318,802.33","316,126.33","99,818.10","24,616.15"


## Incoming

In [10]:
df_in.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1299 entries, 0 to 1298
Data columns (total 28 columns):
 #   Column                                 Non-Null Count  Dtype              
---  ------                                 --------------  -----              
 0   package_row_id                         1299 non-null   object             
 1   delivery_type                          1299 non-null   object             
 2   license_number                         1299 non-null   object             
 3   manifest_number                        1299 non-null   object             
 4   created_date                           1299 non-null   object             
 5   received_datetime                      1299 non-null   datetime64[ns, UTC]
 6   shipper_facility_license_number        1299 non-null   object             
 7   shipper_facility_name                  1299 non-null   object             
 8   recipient_facility_license_number      1299 non-null   object             
 9   recipien

In [16]:
df_in['per_unit_incoming'] = df_in['shipper_wholesale_price'] / df_in['shipped_quantity']

In [62]:
df_in_price = df_in[df_in['shipper_wholesale_price'].notnull()]

In [63]:
average_incoming_package_id = df_in_price.groupby('package_id')['per_unit_incoming'].mean()
df_avg_incoming_price = pd.Series(average_incoming_package_id).to_frame()
df_avg_incoming_price = df_avg_incoming_price.reset_index()

In [64]:
average_incoming_product = df_in.groupby('product_name')['per_unit_incoming'].mean()
df_avg_product = pd.Series(average_incoming_product).to_frame()
df_avg_product = df_avg_product.reset_index()
df_avg_product.rename(columns={'per_unit_incoming':'per_unit_product'}, inplace=True)

## COGS

#### Incoming Price - Package ID

In [65]:
df_cogs = pd.merge(df_sales, df_avg_incoming_price, left_on='tx_package_id', right_on='package_id', how='left')

In [66]:
df_cogs['total_incoming'] = df_cogs['per_unit_incoming'] * df_cogs['tx_quantity_sold']

In [67]:
s_cogs = df_cogs.groupby('year_month')['total_incoming'].sum()
df_cogs_id = pd.Series(s_cogs).to_frame()
df_cogs_id = df_cogs_id.reset_index()

In [68]:
s_cogs_count = df_cogs.groupby('year_month')['total_incoming'].count()
df_cogs_count = pd.Series(s_cogs_count).to_frame()
df_cogs_count = df_cogs_count.reset_index()
df_cogs_count.rename(columns={'total_incoming':'count_incoming'}, inplace=True)

#### Incoming Price - Average Product Price

In [69]:
df_cogs = pd.merge(df_cogs, df_avg_product, left_on='tx_product_name', right_on='product_name', how='left')

In [70]:
df_cogs['total_product'] = df_cogs['tx_quantity_sold'] * df_cogs['per_unit_product']

In [71]:
df_cogs_null = df_cogs[df_cogs['per_unit_incoming'].isnull()]

In [72]:
# df_cogs_null_product = pd.merge(df_cogs_null, df_avg_product, left_on='tx_product_name', right_on='product_name', how='left')
df_cogs_product = df_cogs_null[df_cogs_null['per_unit_product'].notnull()]

In [73]:
product_sum = df_cogs_product.groupby('year_month')['total_product'].sum()
df_product_sum = pd.Series(product_sum).to_frame()
df_product_sum = df_product_sum.reset_index()
df_product_sum.rename(columns={'total_product':'product_sum'}, inplace=True)

In [74]:
product_count = df_cogs_product.groupby('year_month')['total_product'].count()
df_product_count = pd.Series(product_count).to_frame()
df_product_count = df_product_count.reset_index()
df_product_count.rename(columns={'total_product':'product_count'}, inplace=True)

In [75]:
df_cogs_product = pd.merge(df_product_sum, df_product_count)

In [34]:
# cogs_null = df_cogs_null[df_cogs_null['per_unit_product'].isnull()]

In [38]:
# cogs_null['sales_datetime'] = cogs_null['sales_datetime'].dt.strftime("%Y-%m-%d")

C:\Users\Rachel\GitHub\venvs\bespoke\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [39]:
# cogs_null.to_excel('./purple_lotus_import/PL_incoming_null.xlsx')

In [ ]:
# df_cogs[df_cogs['per_unit_incoming'].isnull() & df_cogs['per_unit_product'].notnull()]

## Summary

In [76]:
df_summary = pd.merge(df_revenue, df_cogs_product, how='left')

In [77]:
df_summary = pd.merge(df_summary, df_cogs_id, how='left')

In [78]:
df_summary['product_sum'] = df_summary['product_sum'].fillna(0)

In [79]:
df_summary['product_count'] = df_summary['product_count'].fillna(0)

In [80]:
df_summary['cogs'] = df_summary['total_incoming'] + df_summary['product_sum']

In [81]:
df_summary = pd.merge(df_summary, df_cogs_count)
df_summary = pd.merge(df_summary, df_total_count)

In [82]:
df_summary['total_count_incoming'] = df_summary['product_count'] + df_summary['count_incoming']

In [84]:
df_summary['margin_$'] = df_summary['revenue'] - df_summary['cogs']
df_summary['margin_%'] = df_summary['margin_$'] / df_summary['revenue']

In [85]:
df_summary['coverage'] = df_summary['total_count_incoming'] / df_summary['total_count']

In [86]:
df_summary_simp = df_summary[['year_month', 'revenue', 'cogs', 'margin_$', 'margin_%', 'total_count_incoming', 'total_count', 'coverage']]

In [87]:
df_summary_simp

,year_month,revenue,cogs,margin_$,margin_%,total_count_incoming,total_count,coverage
0,2020-01,"2,244,143.75","453,434.00","1,790,709.75",0.80,44084,85291,0.52
1,2020-02,"2,299,761.12","597,745.07","1,702,016.05",0.74,49738,91822,0.54
2,2020-03,"2,589,911.90","810,062.16","1,779,849.74",0.69,59458,93766,0.63
3,2020-04,"2,432,639.72",NaN,NaN,NaN,63012,81016,0.78
4,2020-05,"2,874,144.58",NaN,NaN,NaN,76263,93213,0.82
5,2020-06,"2,865,584.60","897,627.45","1,967,957.15",0.69,74911,96721,0.77
6,2020-07,"3,108,843.55","808,980.48","2,299,863.07",0.74,71956,110904,0.65
7,2020-08,"3,018,253.05","731,940.63","2,286,312.42",0.76,63141,115236,0.55
8,2020-09,"2,960,355.28","692,645.31","2,267,709.97",0.77,61699,113565,0.54
9,2020-10,"2,929,401.65","643,332.02","2,286,069.63",0.78,59602,112335,0.53


In [78]:
df_summary_simp.to_excel('./Eskar_Holdings_analysis.xlsx')

## Inventory

In [42]:
df_inventory.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23518 entries, 0 to 23517
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   license_number         23518 non-null  object 
 1   package_id             23518 non-null  object 
 2   package_label          23518 non-null  object 
 3   type                   23518 non-null  object 
 4   packaged_date          23518 non-null  object 
 5   package_type           23518 non-null  object 
 6   product_name           23518 non-null  object 
 7   product_category_name  23518 non-null  object 
 8   quantity               23518 non-null  float64
 9   unit_of_measure        23518 non-null  object 
 10  is_testing_sample      23518 non-null  bool   
 11  is_trade_sample        23518 non-null  bool   
 12  is_on_hold             23518 non-null  bool   
 13  archived_date          0 non-null      object 
 14  finished_date          205 non-null    object 
dtypes:

In [43]:
df_inventory_incoming = pd.merge(df_inventory, df_avg_incoming_price, on='package_id', how='left')

In [44]:
df_inv_null = df_inventory_incoming[df_inventory_incoming['per_unit_incoming'].isnull()]

In [45]:
df_inv_product = pd.merge(df_inv_null, df_avg_product, on='product_name', how='left')

In [ ]:
# df_inv_product

In [46]:
df_inv_product_price = df_inv_product[df_inv_product['per_unit_product'].notnull()]

In [47]:
df_inv_product_price['total_price'] = df_inv_product_price['quantity'] * df_inv_product_price['per_unit_product']

C:\Users\Rachel\GitHub\venvs\bespoke\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [48]:
inventory_product_value = df_inv_product_price['total_price'].sum()
print(f'{inventory_product_value:,.2f}')

1,148,676.52


In [49]:
df_inventory_incoming['total_price'] = df_inventory_incoming['quantity'] * df_inventory_incoming['per_unit_incoming']
inventory_value = df_inventory_incoming['total_price'].sum()
print(f'{inventory_value:,.2f}')

inf


In [50]:
total_inv_value = inventory_product_value + inventory_value
print(f'{total_inv_value:,.2f}')

inf


In [51]:
inv_count_product = df_inv_product_price['per_unit_product'].count()

In [52]:
inv_count_product

9459

In [53]:
inv_count_incoming = df_inventory_incoming['per_unit_incoming'].count()
inv_count_total = df_inventory_incoming['quantity'].count()


In [54]:
inv_total_incoming = inv_count_product + inv_count_incoming

In [55]:
inv_total_incoming

10953

In [56]:
inv_count_total

23518

In [57]:
inventory_coverage = inv_total_incoming / inv_count_total
print(f'{inventory_coverage:,.0%}')

47%


# Notes

In [ ]:
# df_in[df_in.duplicated('package_id')]
# df_sales[df_sales.duplicated('tx_package_id')]

In [ ]:
# pd.merge(df_sales, df_in[['package_id','per_unit_incoming']], left_on='tx_package_id', right_on='package_id', how='left')

In [ ]:
# df_sales.merge(df_in['package_id','per_unit_incoming'], left_on='tx_package_id', right_on='package_id', how='left')

In [ ]:
# incoming_transfer_simp = company_incoming_transfer_packages_dataframe[[
# 'license_number',
# 'created_date',
# 'package_id',
# 'product_category_name',
# 'product_name',
# 'shipper_wholesale_price',
# 'shipped_quantity',
# 'shipped_unit_of_measure',
# 'item_unit_weight',
# 'item_unit_weight_unit_of_measure_name']]

In [ ]:
# incoming_transfer_simp

In [ ]:
# outgoing_transfer_simp = company_outgoing_transfer_packages_dataframe[[
# 'license_number',
# 'created_date',
# 'package_id',
# 'product_category_name',
# 'product_name',
# 'shipper_wholesale_price',
# 'shipped_quantity',
# 'shipped_unit_of_measure',
# 'item_unit_weight',
# 'item_unit_weight_unit_of_measure_name']]

In [ ]:
# company_sales_transactions_dataframe.to_csv('hpcc_all_sales_10_13_21.csv')

In [ ]:
# company_sales_transactions_dataframe.info()

In [ ]:
# sales_simp = company_sales_transactions_dataframe[[
# 'license_number',
# 'sales_datetime',
# 'tx_package_id',
# 'tx_product_name',
# 'tx_product_category_name',
# 'tx_unit_of_measure',
# 'tx_quantity_sold',
# 'tx_total_price']]

In [ ]:
# inventory_simp = company_inventory_packages_dataframe[[
# 'license_number',
# 'package_id',
# 'packaged_date',
# 'product_name',
# 'product_category_name',
# 'quantity',
# 'unit_of_measure'
# ]]

In [ ]:
# sales_simp['year_month'] = sales_simp['sales_datetime'].dt.strftime("%Y-%m")

In [ ]:
# sales_simp.groupby(by=['year_month'], as_index=False).count()

In [ ]:
# sales_drop = sales_simp.drop(['sales_datetime', 'month_year'], axis=1)

In [ ]:
# sales_simp.to_csv('./hpcc_sales_2021_10_13.csv')

In [ ]:
# INVENTORY_DATES = [
#     '09/30/2020',
#     '10/31/2020',
#     '11/30/2020',
#     '12/31/2020',
#     '01/31/2021',
#     '02/28/2021',
#     '03/31/2021',
#     '04/30/2021',
#     '05/31/2021',
#     '06/30/2021',
#     '07/31/2021',
#     '08/31/2021',
#     '09/30/2021',
# ]

In [ ]:
# %autoreload 2

# sys.path.append(path.realpath(path.join(os.getcwd(), "../../scripts/analysis")))
# sys.path.append(path.realpath(path.join(os.getcwd(), "../../src")))

# from util import active_inventory_util as util

In [ ]:
# d = util.Download()
# d.download_dataframes(
#     incoming_transfer_packages_dataframe=company_incoming_transfer_packages_dataframe,
#     outgoing_transfer_packages_dataframe=company_outgoing_transfer_packages_dataframe,
#     sales_transactions_dataframe=company_sales_transactions_dataframe,
# )

In [ ]:
# q = util.Query()
# q.inventory_dates = INVENTORY_DATES
# q.company_name = COMPANY_NAME

# id_to_history = util.get_histories(d)
# util.print_counts(id_to_history)
# util.create_inventory_xlsx(id_to_history, q)

In [ ]:
# computed_inventory_package_records = util.create_inventory_dataframe_by_date(id_to_history, INVENTORY_DATE)
# computed_inventory_packages_dataframe = pandas.DataFrame(
#     computed_inventory_package_records,
#     columns=[
#         'package_id',
#         'Arrived Date',
#         'Product Category',
#         'Product Name',
#         'Current Quantity',
#         'Sold Date',
#     ]
# )
# computed_inventory_packages_dataframe

In [ ]:
# date_to_inventory_dataframe = {}
# for date, inventory_records in date_to_inventory_records.items():
#     date_to_inventory_dataframe[date] = pandas.DataFrame(
#         inventory_records,
#         columns=[
#             'package_id',
#             'Arrived Date',
#             'Product Category',
#             'Product Name',
#             'Current Quantity',
#             'Sold Date',
#         ]
#     )
    
# date_to_inventory_dataframe[list(date_to_inventory_dataframe.keys())[0]]

In [ ]:
# raw_incoming_transfer_packages_dataframe = pandas.read_excel('data/20210930/royal_apothecary_incoming_transfer_packages_20200101_20210930.xlsx', header=0)
# len(raw_incoming_transfer_packages_dataframe.index), raw_incoming_transfer_packages_dataframe.columns

In [ ]:
# incoming_transfer_packages_dataframe = raw_incoming_transfer_packages_dataframe[[
#     'date_type',
#     'transfer_row_id',
#     'delivery_row_id',
#     'package_row_id',
#     'delivery_type',
#     'manifest_number',
#     'created_date',
#     'received_datetime',
#     'shipper_facility_license_number',
#     'shipper_facility_name',
#     'recipient_facility_license_number',
#     'recipient_facility_name',
#     'shipment_type_name',
#     'shipment_transaction_type',
#     'package_id',
#     'package_label',
#     'type',
#     'product_category_name',
#     'product_name',
#     'shipper_wholesale_price',
#     'shipped_quantity',
#     'package_lab_results_status',
#     'shipment_package_state',
#     'is_testing_sample',
#     'is_trade_sample'
# ]]

In [ ]:
# for date, inventory_dataframe in date_to_inventory_dataframe.items():
#     print(date)
#     print(f'# of packages in inventory: {len(inventory_dataframe.index)}')

#     inventory_with_incoming_transfer_packages_dataframe = inventory_dataframe.astype({'package_id': 'int64'}).merge(incoming_transfer_packages_dataframe, on='package_id', how='inner', suffixes=('_l', '_r'))
# #     print(f'# of packages in inventory with incoming package: {len(inventory_with_incoming_transfer_packages_dataframe.index)}')
    
#     inventory_with_cost_records = inventory_with_incoming_transfer_packages_dataframe.to_dict('record')

#     total_valuation_cost = 0
#     import math
#     for inventory_with_cost_record in inventory_with_cost_records:
#         incoming_shipped_price = inventory_with_cost_record['shipper_wholesale_price']
#         if math.isnan(incoming_shipped_price):
#             incoming_shipped_price = 0
#         incoming_shipped_quantity = inventory_with_cost_record['shipped_quantity']
#         current_quantity = inventory_with_cost_record['Current Quantity']
#         total_valuation_cost += float(current_quantity) * (incoming_shipped_price / incoming_shipped_quantity)

#     print(f'Inventory valuation (based on COST): ${total_valuation_cost}')
#     print('')

In [ ]:
# import json
# import pandas
# import numpy
# import os
# import sys
# from os import path